In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
def load_txt_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    coords = []
    for line in lines:
        left_coord, right_coord = line.strip().split('),(')
        left_x, left_y = left_coord.strip('() ').split(',')
        right_x, right_y = right_coord.strip('() ').split(',')
        coords.append(((int(left_x), int(left_y)),
                      (int(right_x), int(right_y))))
    return coords


In [3]:
def preprocess_dataset(punch_dir, no_punch_dir):
    punch_data = []
    no_punch_data = []

    # Load punch data
    for i in range(1, 251):
        file_path = os.path.join(punch_dir, f"punch_coordinates{i}.txt")
        elbow_coords = load_txt_file(file_path)
        punch_data.extend(elbow_coords)

    for i in range(1, 251):
        file_path = os.path.join(no_punch_dir, f"no_punch_coordinates{i}.txt")
        elbow_coords = load_txt_file(file_path)
        no_punch_data.extend(elbow_coords)

    x = np.array(punch_data + no_punch_data)
    y = np.array([1] * len(punch_data) + [0] * len(no_punch_data))

    return x, y


In [4]:
def build_model(max_length):
    model = Sequential()
    model.add(Flatten(input_shape=(max_length, 2)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [5]:
punch_dir = "dataset/punch"
no_punch_dir = "dataset/no_punch"


In [6]:
x, y = preprocess_dataset(punch_dir, no_punch_dir)


In [7]:
max_length = max([len(coords) for coords in x])
x = pad_sequences(x, maxlen=max_length, dtype='float32', padding='post')


In [8]:
model = build_model(max_length)


In [9]:
model.fit(x, y, epochs=10, batch_size=32)


Epoch 1/10
953/953 [==============================] - 1s 827us/step - loss: 0.9098 - accuracy: 0.7368
Epoch 2/10
953/953 [==============================] - 1s 783us/step - loss: 0.5254 - accuracy: 0.7749
Epoch 3/10
953/953 [==============================] - 1s 744us/step - loss: 0.5110 - accuracy: 0.7791
Epoch 4/10
953/953 [==============================] - 1s 736us/step - loss: 0.5283 - accuracy: 0.7815
Epoch 5/10
953/953 [==============================] - 1s 743us/step - loss: 0.5152 - accuracy: 0.7830
Epoch 6/10
953/953 [==============================] - 1s 766us/step - loss: 0.5068 - accuracy: 0.7850
Epoch 7/10
953/953 [==============================] - 1s 742us/step - loss: 0.5036 - accuracy: 0.7861
Epoch 8/10
953/953 [==============================] - 1s 743us/step - loss: 0.5171 - accuracy: 0.7859
Epoch 9/10
953/953 [==============================] - 1s 756us/step - loss: 0.4970 - accuracy: 0.7877
Epoch 10/10
953/953 [==============================] - 1s 752us/step - loss: 0.468

In [10]:
def predict_punch(file_path, model, max_length):
    elbow_coords = load_txt_file(file_path)
    x = np.array([elbow_coords])
    x = x.reshape(-1, 2, 2)
    x = pad_sequences(x, maxlen=max_length, dtype='float32', padding='post')
    prediction = model.predict(x)
    likelihood = prediction[0][0]
    if likelihood >= 0.5:
        print("a punch.")
    else:
        print("not a punch.")


In [11]:
single_punch_file = "./dataset/no_punch/no_punch_coordinates20.txt"
predict_punch(single_punch_file, model, max_length)


3/3 [==============================] - 0s 1000us/step
not a punch.


In [14]:
single_punch_file = "./dataset/punch/punch_coordinates21.txt"
predict_punch(single_punch_file, model, max_length)


2/2 [==============================] - 0s 1ms/step
a punch.
